In [ ]:
!git clone https://github.com/facebookresearch/sam2.git sam2_repo

Cloning into 'sam2_repo'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 128.11 MiB | 44.53 MiB/s, done.
Resolving deltas: 100% (381/381), done.


In [ ]:
%cd sam2_repo

/content/sam2_repo


In [ ]:
!git checkout 2b90b9f5ceec907a1c18123530e92e794ad901a4

Note: switching to '2b90b9f5ceec907a1c18123530e92e794ad901a4'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 2b90b9f remove `.pin_memory()` in `obj_pos` of `SAM2Base` to resolve and error in MPS (#495)


In [ ]:
!pip install -e .[dev] -q

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.9 MB/s eta 0

In [ ]:
!cd ./checkpoints && ./download_ckpts.sh

--2025-08-15 04:35:20--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_tiny.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.171.22.13, 3.171.22.118, 3.171.22.68, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.171.22.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156008466 (149M) [application/vnd.snesdev-page-table]
Saving to: ‘sam2.1_hiera_tiny.pt’

sam2.1_hiera_tiny.p 100%[===================>] 148.78M   238MB/s    in 0.6s    

2025-08-15 04:35:21 (238 MB/s) - ‘sam2.1_hiera_tiny.pt’ saved [156008466/156008466]

--2025-08-15 04:35:21--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_small.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.171.22.13, 3.171.22.118, 3.171.22.68, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.171.22.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184416285 (176M) [application/vn

In [ ]:
!mkdir -p sam2/configs
!cp -v /content/config/train.yaml sam2/configs/train.yaml

'/content/config/train.yaml' -> 'sam2/configs/train.yaml'


In [ ]:
!python training/train.py -c 'configs/train.yaml' --use-cluster 0 --num-gpus 1

###################### Train App Config ####################
scratch:
  resolution: 1024
  train_batch_size: 1
  num_train_workers: 10
  num_frames: 1
  max_num_objects: 3
  base_lr: 5.0e-06
  vision_lr: 3.0e-06
  phases_per_epoch: 1
  num_epochs: 30
dataset:
  img_folder: /content/drive/MyDrive/data/sam2_data/samples
  gt_folder: /content/drive/MyDrive/data/sam2_data/samples
  multiplier: 2
vos:
  train_transforms:
  - _target_: training.dataset.transforms.ComposeAPI
    transforms:
    - _target_: training.dataset.transforms.RandomHorizontalFlip
      consistent_transform: true
    - _target_: training.dataset.transforms.RandomAffine
      degrees: 25
      shear: 20
      image_interpolation: bilinear
      consistent_transform: true
    - _target_: training.dataset.transforms.RandomResizeAPI
      sizes: ${scratch.resolution}
      square: true
      consistent_transform: true
    - _target_: training.dataset.transforms.ColorJitter
      consistent_transform: true
      brightness:

In [ ]:
#Mast Generator w/ a prompt
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
import os
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
if torch.cuda.get_device_properties(0).major >= 8:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

checkpoint = "./sam2_logs/configs/train.yaml/checkpoints/checkpoint.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_b+.yaml"
sam2 = build_sam2(model_cfg, checkpoint, device="cuda")
predictor = SAM2ImagePredictor(sam2)


# Process all imgs

In [ ]:
import os
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm


image_dir = "/content/drive/MyDrive/data/sam2_data/samples"
output_zero_dir = "/content/drive/MyDrive/final/output/masked/zero_out"
output_crop_dir = "/content/drive/MyDrive/final/output/masked/crop"
output_mask_dir = "/content/drive/MyDrive/final/output/masks"

os.makedirs(output_zero_dir, exist_ok=True)
os.makedirs(output_crop_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)



input_box = np.array([44, 102, 1016, 814])  # (x_min, y_min, x_max, y_max)
input_point = np.array([[523, 686]])
input_label = np.array([1])  # foreground


def zero_out_background(img_bgr, mask_bool):
    out = img_bgr.copy()
    out[~mask_bool] = 0
    return out

def tight_crop(img_bgr, mask_bool):
    ys, xs = np.where(mask_bool)
    if xs.size == 0 or ys.size == 0:
        return img_bgr
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    return img_bgr[y0:y1+1, x0:x1+1]




In [ ]:
image_list = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".png"))]

for img_name in tqdm(image_list, desc="Processing images"):
    img_path = os.path.join(image_dir, img_name)
    img_rgb = np.array(Image.open(img_path).convert("RGB"))


    predictor.set_image(img_rgb)


    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        box=input_box,
        multimask_output=False
    )

    mask_bool = masks[0] > 0

    # Save: Masks
    mask_img = (mask_bool.astype(np.uint8) * 255)
    mask_name = os.path.splitext(img_name)[0] + "_mask.png"
    mask_path = os.path.join(output_mask_dir, mask_name)
    Image.fromarray(mask_img).save(mask_path)

    # Save: Zero-out
    zero_img = zero_out_background(img_rgb, mask_bool)
    zero_path = os.path.join(output_zero_dir, img_name)
    Image.fromarray(zero_img).save(zero_path)

    # Save: Crop
    crop_img = tight_crop(img_rgb, mask_bool)
    crop_path = os.path.join(output_crop_dir, img_name)
    Image.fromarray(crop_img).save(crop_path)

print(f"Saved Zero-out Path: {output_zero_dir}")
print(f"Saved Crop Path: {output_crop_dir}")


Processing images: 100%|██████████| 57/57 [00:06<00:00,  8.58it/s]

Saved Zero-out Path: /content/drive/MyDrive/final/output/masked/zero_out
Saved Crop Path: /content/drive/MyDrive/final/output/masked/crop
